# Tutorial con la ecuación de Lindblad y QMC
$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$
$\renewcommand{\braket}[1]{\left\langle{#1}\right\rangle}$

Artículo: https://arxiv.org/pdf/1906.04478.pdf

Documentación: https://qutip.org/documentation.html


Se prueban dinámicas de sistemas cuánticos con dos métodos distintos: la integración de la ecuación de Lindblad y el método de Monte Carlo cuántico.

En primer lugar, se importan los paquetes que se van a usar en el programa. Del paquete \texttt{matplotlib} importamos la función \texttt{pyplot}, renombrándola como \texttt{plt} cuando la usemos.

In [52]:
import matplotlib.pyplot as plt

Importamos \texttt{numpy} como \texttt{np}.

In [53]:
import numpy as np

De \texttt{qutip} lo importamos todo, simbolizado con el asterisco (\texttt{*}).

In [54]:
from qutip import *

Como en el artículo, se realiza con un sistema de dos niveles, cuyo hamiltoniano es el siguiente
    
$$H = E \ket{1}\bra{1} + \Omega\left(\ket{0}\bra{1} + \ket{1}\bra{0}  \right).$$

Tomamos los parámetros del artículo, además de crear una base de los vectores $\ket{0}$ y $\ket{1}$.

In [55]:
N = 2 # Dimensión del espacio de Hilbert

psi0 = basis(N,0) # Vector |0>
psi1 = basis(N,1) # Vector |1>

Definición del hamiltoniano como una función. Esto se hace con \texttt{lambda}, seguido de los parámetros de los que dependerá. Tras los dos puntos, escribimos la forma de la función que estamos definiendo.

In [56]:
Hfun = lambda E, omega: E*psi1*psi1.dag() + omega*(psi0*psi1.dag() + psi1*psi0.dag()) 

El vector inicial será $\ket{1}$. Hacemos la simulación sin ningún término disipativo. Dichos términos se introducen en \texttt{c_op_list}, con formato de lista, con la función \texttt{append}.

In [57]:
# Definición de los parámetros y evaluación en la función que define al Hamiltoniano
omega = 1.0   
E = 1.0
H = Hfun(E, omega)

# Parámetros de la simulación: vector (o matriz densidad) inicial, vector de tiempos y lista de operadores de Lindblad
psiini = basis(N,1)   # |1>
tlist = np.linspace(0,5,1000) # Vector de tiempos
c_op_list = [] # Sin disipacion

Para el caso del Monte Carlo Cuántico, tenemos que definir también un número de trayectorias

In [58]:
ntraj = 100

Realizamos las evoluciones temporales (https://qutip.org/docs/latest/guide/guide-dynamics.html). Vamos a usar tres funciones distintas:

#### sesolve

Resuelve la ecuación de Schrödinger para un vector o una matriz unitaria para un hamiltoniano dado. Útil si no hay términos disipativos.

#### mesolve

Resuelve la ecuación de Lindblad para un vector dado o una matriz densidad. Se pueden de aportar (o no) los operadores de la parte disipativa.

#### mcsolve

Algoritmo de Monte Carlo Cuántico para un vector de estado, un hamiltoniano dado y unos operadores que marcan la disipación. Si estos no son incluidos, se llama directamente a la función sesolve. Hemos de dar el número de trayectorias.


## Caso sin disipación

Resolvemos el caso sin disipación para la Ecuación de Lindblad. Vamos a indicar que nos promedie $\ket{0}\bra{0}$ y $\ket{1}\bra{1}$.

In [59]:
me = mesolve(H, psiini, tlist, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()])

Se crea una clase, \texttt{me}, donde podemos acceder a distintos parámetros. Si no indicamos nada para promediar, podemos acceder a la evolución del estado.

In [60]:
me = mesolve(H, psiini, tlist, c_op_list)
me.states

Para ver los valores esperados, hacemos lo siguiente:

In [61]:
me = mesolve(H, psiini, tlist, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()])

me.expect[0] # Primer elemento de la lista

In [62]:
me.expect[1] # Segundo elemento de la lista

Hacemos lo propio con Monte Carlo, indicando al final el número de trayectorias.

In [63]:
mc = mcsolve(H, psiini, tlist, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()], ntraj)

En la representación gráfica señalaremos de color magenta el valor de $\ket{0}\bra{0}$ obtenido con QMC (en este caso concreto sesolve); amarillo para $\ket{1}\bra{1}$ y finalmente línea negra de puntos para comparar con la ecuación maestra.

In [64]:
plt.plot(tlist, mc.expect[0],'m',lw=2)
plt.plot(tlist, mc.expect[1],'y',lw=2)
plt.plot(tlist, me.expect[0],'k--',lw=1.5)
plt.plot(tlist, me.expect[1],'k--',lw=1.5)
plt.xlabel('Tiempo',fontsize=14)
plt.ylabel('Poblaciones', fontsize=14)
plt.legend(('Estado ${0}$','Estado ${1}$', 'ME'))

## Caso con disipación

Vamos a introducir la disipación. Para ello, los operadores de Lindblad serán:

$$\sqrt{\gamma (1+n)} \sigma^+,$$
$$\sqrt{\gamma n} \sigma^-.$$


In [41]:
# Definición de los parámetros y de los operadores de Lindblad

gamma = 0.1
avg_n = 1.0



Aumentamos el número de trayectorias y el valor del tiempo final.

In [47]:
tlist2 = np.linspace(0,10,1000)
ntraj2 = 500

Definimos los operadores como lista y los añadimos con la función \texttt{append} para posteriormente introducirlos como tal en la función. 

In [48]:
c_op_list = []
c_op_list.append(np.sqrt(gamma * (1 + avg_n)) * sigmap())
c_op_list.append(np.sqrt(gamma * avg_n) * sigmam())

Notemos que podríamos haber hecho lo propio con los operadores que buscamos promediar (se introducen como lista), en este caso introducidos directamente en la función.

In [49]:
me2 = mesolve(H, psiini, tlist2, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()])

In [50]:
mc2 = mcsolve(H, psiini, tlist2, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()], ntraj2)

In [51]:
plt.plot(tlist2, mc2.expect[0],'m',lw=2)
plt.plot(tlist2, mc2.expect[1],'y',lw=2)
plt.plot(tlist2, me2.expect[0],'k--',lw=1.5)
plt.plot(tlist2, me2.expect[1],'k--',lw=1.5)
plt.xlabel('Tiempo',fontsize=14)
plt.ylabel('Poblaciones', fontsize=14)
plt.legend(('Estado ${0}$','Estado ${1}$', 'ME'))

## Caso con disipación y sin segundo término del hamiltoniano $(\Omega = 0)$. 

Creamos un nuevo hamiltoniano haciendo nulo el parámetro $\Omega$ introdicido previamente.

In [20]:
H3 = Hfun(E, 0)
tlist3 = np.linspace(0,20,2000)
ntraj3 = 1000

In [21]:
me3 = mesolve(H3, psiini, tlist3, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()])

In [22]:
mc3 = mcsolve(H3, psiini, tlist3, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()], ntraj2)

In [23]:
plt.plot(tlist3, mc3.expect[0],'m',lw=2)
plt.plot(tlist3, mc3.expect[1],'y',lw=2)
plt.plot(tlist3, me3.expect[0],'k--',lw=1.5)
plt.plot(tlist3, me3.expect[1],'k--',lw=1.5)
plt.xlabel('Tiempo',fontsize=14)
plt.ylabel('Poblaciones', fontsize=14)
plt.legend(('Estado ${0}$','Estado ${1}$', 'ME'))


Otra opción interesante es cambiar el número de trayectorias. Simplemente, añadimos una lista con las trayectorias que deseemos para la simulación en lugar de un único parámetro.

In [24]:
ntraj_list = [1, 10, 100, 1000, 5000] # list of number of trajectories to avg. over

In [25]:
mc_list = mcsolve(H3, psiini, tlist3, c_op_list, [psi0*psi0.dag(),psi1*psi1.dag()], ntraj_list)

In [26]:
fig, axes = plt.subplots(5, 1, sharex=True, figsize=(8,14))


for idx, n in enumerate(ntraj_list):

    axes[idx].step(tlist3, mc_list.expect[idx][0], 'm', lw=2)
    axes[idx].step(tlist3, mc_list.expect[idx][1], 'y', lw=2)
    
    axes[idx].set_yticks(np.linspace(0, 2, 5))
    axes[idx].set_ylim([-0.25, 1.5])
    axes[idx].set_ylabel('Poblaciones - %d trajs' % n, fontsize=14)
    
    axes[idx].legend(('Estado ${0}$','Estado ${1}$'))
